In [1]:
import tensorflow as tf
from tensorflow import keras
import ipywidgets

In [2]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.test.is_built_with_cuda()
print(tf.version.VERSION)
import sys
sys.version

Num GPUs Available:  1
2.12.0


'3.9.16 (main, Dec  7 2022, 01:11:51) \n[GCC 9.4.0]'

In [ ]:
# ! pip install opencv-python

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Imports

In [4]:
import os
import io
import matplotlib.pyplot as plt
import cv2
import imageio
#import medmnist
#import ipywidgets
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint
import cv2 as cv
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Dense
from keras.regularizers import l2
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.regularizers import l2

# Setting seed for reproducibility
SEED = 42
os.environ["TF_CUDNN_DETERMINISTIC"] = "1"
np.random.seed(SEED)

## Hyperparameters

The hyperparameters are chosen via hyperparameter
search. You can learn more about the process in the "conclusion" section.

In [5]:
#Binray

# DATA
DATASET_NAME = "LAD2000"
BATCH_SIZE = 32
PATH = '/content/gdrive/MyDrive/Datasets/LAD-2000'
# AUTO = tf.data.AUTOTUNE
INPUT_SHAPE = (75, 75,14, 2)
NUM_CLASSES = 2

# OPTIMIZER
LEARNING_RATE = 1e-2
WEIGHT_DECAY = 1e-5

# TRAINING
EPOCHS = 100

# TUBELET EMBEDDING
PATCH_SIZE = (8, 8, 8)
NUM_PATCHES = (INPUT_SHAPE[0] // PATCH_SIZE[0]) ** 2

# ViViT ARCHITECTURE
LAYER_NORM_EPS = 1e-6
PROJECTION_DIM = 128
NUM_HEADS = 8
NUM_LAYERS = 8

In [ ]:
#binary_data_path = '/content/gdrive/MyDrive/Datasets/UCF-original/Binary_Classification/Final_Binary_Data/'
#X_train = np.concatenate((np.load(binary_data_path + 'Train/Non_violence/non_X_train.npy'), np.load(binary_data_path + 'Train/Violence/V_X_train.npy')))

#Load optical flow data

In [6]:
X_train_opt = np.load('/content/gdrive/MyDrive/All extracted frames/LAD2000-opticalflow/Train/X_flow.npy',allow_pickle=True)
y_train_opt = np.load('/content/gdrive/MyDrive/All extracted frames/LAD2000-opticalflow/Train/y_flow.npy', allow_pickle=True)

In [7]:
X_val_opt = np.load('/content/gdrive/MyDrive/All extracted frames/LAD2000-opticalflow/Val/X_flow.npy',allow_pickle=True)
y_val_opt = np.load('/content/gdrive/MyDrive/All extracted frames/LAD2000-opticalflow/Val/y_flow.npy', allow_pickle=True)

In [8]:
X_test_opt = np.load('/content/gdrive/MyDrive/All extracted frames/LAD2000-opticalflow/Test/X_flow.npy',allow_pickle=True)
y_test_opt = np.load('/content/gdrive/MyDrive/All extracted frames/LAD2000-opticalflow/Test/y_flow.npy', allow_pickle=True)

#Optical-flow calculation


In [ ]:
import cv2
from google.colab.patches import cv2_imshow
def opticalflow(X_data, y_data):
    
    optical_flow = []
    flow_labels = []
    for i in range (X_data.shape[0]):
        packet = []
        if i == 1487:
            print(i)
        for j in range(1, X_data.shape[1]):
            frame1 = X_data[i][j-1]
            frame2 = X_data[i][j]
            gray1 = cv2.cvtColor((255*frame1).astype(np.uint8), cv2.COLOR_BGR2GRAY)
            gray2 = cv2.cvtColor((255*frame2).astype(np.uint8), cv2.COLOR_BGR2GRAY)
            flow = cv2.calcOpticalFlowFarneback(gray1, gray2, None, 0.5, 3, 15, 3, 5, 1.2, 0)
            packet.append(flow)
        optical_flow.append(np.asarray(packet))
        flow_labels.append(y_data[i])

            # # display the optical flow field
            # h, w = gray1.shape
            # y, x = np.mgrid[0:h:5, 0:w:5].reshape(2, -1)
            # fx, fy = flow[y, x].T
            # lines = np.vstack([x, y, x + fx, y + fy]).T.reshape(-1, 2, 2)
            # lines = np.int32(lines + 0.5)
            # vis = cv2.cvtColor(gray2, cv2.COLOR_GRAY2BGR)
            # cv2.polylines(vis, lines, 0, (0, 255, 0))
            # for (x1, y1), (_x2, _y2) in lines:
            #     cv2.circle(vis, (x1, y1), 1, (0, 255, 0), -1)
            # cv2_imshow(vis)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()

    return np.asarray(optical_flow), np.asarray(flow_labels)

In [9]:
X_train_opt.shape

(4476, 14, 75, 75, 2)

In [10]:
X_train_opt = np.moveaxis(X_train_opt, 1, 3)
X_val_opt = np.moveaxis(X_val_opt, 1, 3)
X_test_opt = np.moveaxis(X_test_opt, 1, 3)

In [11]:
print(X_train_opt.shape, X_val_opt.shape, X_test_opt.shape)

(4476, 75, 75, 14, 2) (1002, 75, 75, 14, 2) (651, 75, 75, 14, 2)


In [12]:
dim = X_train_opt.shape
dim = dim[1:]
print(dim)

(75, 75, 14, 2)


In [13]:
y_train_opt[7]

0.0

In [14]:
X_train_opt = np.asarray(X_train_opt).astype(np.float32)
X_val_opt = np.asarray(X_val_opt).astype(np.float32)
X_test_opt = np.asarray(X_test_opt).astype(np.float32)

In [15]:
y_train_opt = np.asarray(y_train_opt).astype(np.float32)
y_val_opt = np.asarray(y_val_opt).astype(np.float32)
y_test_opt = np.asarray(y_test_opt).astype(np.float32)

In [15]:
# DATA
#DATASET_NAME = "UCF-Crime"
BATCH_SIZE = 32
#PATH = '/content/gdrive/MyDrive/Datasets/UCF-original/Action-Recognition/labelled_frames/'
#PATH = '/content/gdrive/MyDrive/Datasets/UCF-original/Binary_Classification/Final_Binary_Data/'
# AUTO = tf.data.AUTOTUNE
INPUT_SHAPE = (75, 75, 15, 3)
NUM_CLASSES = 2
# NUM_CLASSES = 2

# OPTIMIZER
LEARNING_RATE = 1e-2
WEIGHT_DECAY = 1e-5

# TRAINING
EPOCHS = 100

# TUBELET EMBEDDING
PATCH_SIZE = (8, 8, 8)
NUM_PATCHES = (INPUT_SHAPE[0] // PATCH_SIZE[0]) ** 2

# ViViT ARCHITECTURE
LAYER_NORM_EPS = 1e-6
PROJECTION_DIM = 128
NUM_HEADS = 8
NUM_LAYERS = 8

In [ ]:
# train_videos, train_labels = np.load(PATH + 'Train/X.npy'), np.load(PATH + 'Train/y.npy')
# val_videos, val_labels = np.load(PATH + 'Val/X.npy'), np.load(PATH + 'Val/y.npy')
# test_videos, test_labels = np.load(PATH + 'Test/X.npy'), np.load(PATH + 'Test/y.npy')

In [ ]:
# train_labels -= 1
# val_labels -= 1
# test_labels -= 1

In [ ]:
# print(train_videos.shape)
# print(train_labels.shape)

(4873, 15, 75, 75, 3)
(4873,)


In [ ]:
# np.unique(train_labels)

array([0, 1])

#Binary with MM model


In [16]:
import os
import sys
import cv2
import zipfile
import numpy as np
import imageio
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn import svm
from sklearn.model_selection import train_test_split
# from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths
from tensorflow.keras import layers, models

In [17]:


model = models.Sequential()
# model.add(layers.BatchNormalization())
model.add(layers.Conv3D(32, kernel_size=(3, 3, 1), activation='relu', kernel_initializer='he_uniform', data_format='channels_last', input_shape=dim))
model.add(layers.MaxPooling3D((2, 2, 1)))
# model.add(layers.BatchNormalization())
model.add(layers.Conv3D(32, kernel_size=(3, 3, 2), activation='relu', kernel_initializer='he_uniform'))
model.add(layers.MaxPooling3D((3, 3, 1)))
# model.add(layers.BatchNormalization())
model.add(layers.Conv3D(32, kernel_size=(3, 3, 1), activation='relu', kernel_initializer='he_uniform'))
model.add(layers.MaxPooling3D((2, 2, 1)))
# model.add(layers.BatchNormalization())
model.add(layers.Flatten())
# model.add(layers.Dropout(0.5))
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dense(1024, activation='relu'))
# model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(16, activation='relu'))
# model.add(layers.BatchNormalization())
model.add(layers.Dense(1, activation='sigmoid'))



In [18]:


model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.00025),
              metrics=['accuracy'])



In [19]:

history = model.fit(X_train_opt, y_train_opt,
                    batch_size=1,
                    epochs=50,
                    validation_data=(X_val_opt, y_val_opt),
                    shuffle=True)


Epoch 1/50
4476/4476 [==============================] - 75s 13ms/step - loss: 0.7132 - accuracy: 0.4969 - val_loss: 0.6934 - val_accuracy: 0.4960
Epoch 2/50
4476/4476 [==============================] - 58s 13ms/step - loss: 0.7227 - accuracy: 0.5273 - val_loss: 0.6892 - val_accuracy: 0.5699
Epoch 3/50
4476/4476 [==============================] - 59s 13ms/step - loss: 0.6943 - accuracy: 0.5496 - val_loss: 0.6842 - val_accuracy: 0.5928
Epoch 4/50
4476/4476 [==============================] - 58s 13ms/step - loss: 0.7022 - accuracy: 0.5060 - val_loss: 0.6923 - val_accuracy: 0.5828
Epoch 5/50
4476/4476 [==============================] - 58s 13ms/step - loss: 0.6929 - accuracy: 0.5369 - val_loss: 0.6889 - val_accuracy: 0.5309
Epoch 6/50
4476/4476 [==============================] - 61s 14ms/step - loss: 0.6775 - accuracy: 0.5717 - val_loss: 0.6919 - val_accuracy: 0.5349
Epoch 7/50
4476/4476 [==============================] - 58s 13ms/step - loss: 0.6818 - accuracy: 0.5943 - val_loss: 0.6709 -